In [11]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import glob
import gzip
import tensorflow as tf
from tensorflow import keras
import time

import matplotlib.pyplot as plt

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

import pickle
import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import pickle
import emoji
from wordcloud import WordCloud
from sklearn.naive_bayes import MultinomialNB

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Meghna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
covid_files_path = r'/Users/Meghna/Desktop/covid_data/covid_tweets.json'

covid_train_data = []
noncovid_train_data = []
covid_train_labels = []
noncovid_train_labels = []

covid_train_data2 = []
noncovid_train_data2 = []


try:
    with open(covid_files_path,'r') as fin:
        for line in fin:
            tweet = json.loads(line)
            covid_train_data.append(tweet['text'])
            covid_train_labels.append(1)
except:
    print("Faulty file ")

In [3]:
noncovid_files_path = r'/Users/Meghna/Desktop/noncovid_data/*'
noncovid_files = glob.glob(noncovid_files_path)
num_noncovid_tweets = 0

for i in range(len(noncovid_files)):
    print("opening file", noncovid_files[i])
    try:
        with gzip.open(noncovid_files[i],'r') as fin:
            for line in fin:
                if num_noncovid_tweets < 27068:
                    tweet = json.loads(line)
                    noncovid_train_data.append(tweet['text'])
                    noncovid_train_labels.append(0)
                    num_noncovid_tweets += 1

    except:
        print("Faulty file ", noncovid_files[i])

opening file /Users/Meghna/Desktop/noncovid_data/noncovid_tweets_oct.gz
opening file /Users/Meghna/Desktop/noncovid_data/noncovid_tweets_nov.gz


In [4]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    lower_case = stripped.lower()
    lower_case = emoji.demojize(lower_case, language='en')
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [5]:
for tweet in covid_train_data:
    covid_train_data2.append(tweet_cleaner_updated(tweet))
    
for ntweet in noncovid_train_data:
    noncovid_train_data2.append(tweet_cleaner_updated(ntweet))

train_data = covid_train_data2 + noncovid_train_data2
train_labels = covid_train_labels + noncovid_train_labels

/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://t.co/NSl7vPwMob
SCUMBAG." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


In [6]:
train_corpus,test_corpus,train_labels,test_labels = train_test_split(train_data,train_labels,stratify=train_labels,test_size=0.25,random_state=1)
train_texts, train_labels, test_texts, test_labels = train_corpus, train_labels, test_corpus, test_labels
x_train, y_train, x_validation, y_validation = train_corpus,train_labels,test_corpus,test_labels

In [7]:
# def accuracy_summary3(pipeline, x_train, y_train, x_val,y_val,x_test, y_test):
#     if y_val_len0 / (len(x_test)*1.) > 0.5:
#         null_accuracy =  y_val_len1/ (len(x_test)*1.)
#     else:
#         null_accuracy = 1. - (y_val_len1 / (len(x_test)*1.))
#     #t0 = time.time()
#     sentiment_fit = pipeline.fit(x_train, y_train)
#     pickle.dump(sentiment_fit, open('/Users/Meghna/Desktop/covid_noncovid_model.sav', 'wb'))
#     #print(classification_report(y_true, y_pred))
#     y_pred = sentiment_fit.predict(x_val)
#     #train_test_time = time() - t0
#     accuracy = accuracy_score(y_val, y_pred)
    
#     print ("null accuracy: {0:.2f}%".format(null_accuracy*100))
#     print ("accuracy score: {0:.2f}%".format(accuracy*100))
#     if accuracy > null_accuracy:
#         print ("model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100))
#     elif accuracy == null_accuracy:
#         print ("model has the same accuracy with the null accuracy")
#     else:
#         print ("model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100))
#     #print ("train and test time: {0:.2f}s".format(train_test_time))
    
#     y_true, y_pred_test = y_test, sentiment_fit.predict(x_test)
#     print(classification_report(y_true, y_pred_test))
#     print ("-"*80)
#     return accuracy

In [14]:
# eclf6 = MultinomialNB()

# for clf, label in zip([eclf6], ['Multinomial NB']):
#     checker_pipeline = Pipeline([
#             ('vectorizer', TfidfVectorizer(stop_words = 'english',max_features=30000,ngram_range=(1, 3))),
#             ('classifier', clf)
#         ])
#     print ("Validation result for {}".format(label))
#     print (clf)
#     clf_accuracy = accuracy_summary3(checker_pipeline, x_train, y_train, x_validation, y_validation, x_test2, y_test2)

Validation result for Multinomial NB
MultinomialNB()


NameError: name 'x_test2' is not defined

In [16]:
t = TfidfVectorizer(stop_words = 'english',max_features=30000,ngram_range=(1, 3))
features = t.fit_transform(x_train)

In [ ]:
# for test features

features_test = t.transform(x_test)

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)